In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType, AgentExecutor, create_openai_functions_agent
#from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.google_search import GoogleSearchResults
import warnings
warnings.filterwarnings(action='ignore')
load_dotenv()

True

In [2]:
llm = AzureChatOpenAI(
    deployment_name=os.environ["AZURE_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_type=os.environ["OPENAI_API_TYPE"],
    temperature=0
) 

In [3]:
template = "What is the leadership url of {company}"
prompt = PromptTemplate.from_template(template)

chain = prompt | llm
chain.invoke({"company": "FactSet"})

AIMessage(content='The leadership URL of FactSet is https://www.factset.com/company/leadership.')

In [5]:
os.environ["GOOGLE_CSE_ID"] = os.environ["GOOGLE_CSE_ID"]
os.environ["GOOGLE_API_KEY"] = os.environ["GOOGLE_API_KEY"]

In [27]:
from langchain.tools import Tool
from langchain_community.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper(k=5)

tool = Tool(
    name="Google Search",
    description="Search Google for recent results.",
    func=search.run,
)

In [20]:
#search.run("what is the weather in Hyderabad")

In [29]:
url = LLMChain(llm=llm, prompt=prompt)

In [30]:
tools = [Tool.from_function(func = search.run, name="URL", description="Useful to find url for companies")]

In [31]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [34]:
agent.invoke({"input": "As an engineer I would like to crawl www.factset.com to identify leadership url"})



> Entering new AgentExecutor chain...
I should use the URL tool to find the URL for FactSet's leadership page.
Action: URL
Action Input: FactSet leadership page
Observation: Meet the Team · Philip Snow · Linda Huber · Helen Shan · Goran Skoko · Robert Robie · Ali van Nes · Catrina Harding · Christopher Ellis. Page 1. Index Methodology Guide for the. FactSet Japan Global Leaders ESG IndexTM. Version 1.0 – May 7, 2021. Page 2. Table of Contents. Index Methodology ... Apr 29, 2020 ... leadership team. We hold our leaders accountable for measuring ... Page 13-15. Page 21. www.factset.com. Page 1. Index Methodology Guide for the. FactSet Japan Mid & Small Cap Leaders ESG IndexTM. Version 1.0 – October 27, 2021. Page 2. Table of Contents. Basics. Content Category: Human Capital. Open:FactSet Page: Click here. Setup a Demo and Access Today: sales@factset.com. Equilar's Board Services data feed ...
Thought:I have found the leadership page for FactSet on their website.
Thought: I now know the

{'input': 'As an engineer I would like to crawl www.factset.com to identify leadership url',
 'output': "The URL for FactSet's leadership page is www.factset.com."}